In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

    
    
    
def Belief(State,p,r):
   
    equilibrium = r/(1-p+r);
    
    if State[0]== 0 and State[1]> 0:
            Belief = (r-r*(p-r)**(State[1]))/(1+r-p);
    elif State[0]== 1 and State[1]> 0:
            Belief = (r+(1-p)*(p-r)**(State[1]))/(1+r-p);
            
    else:
        
        Belief = equilibrium;

    return Belief;

def update(B,C,Reward,alpha,p,r,q,s,gamma,N):
    
    BeliefVector= [0.]*N;
    Y= [0.]*N;
    Action = [0]*N;
    Threshold = 0.;
    Seed = np.random.rand(N);
    
    for i in range(int(N*gamma)):
        Y[i]= Belief(B[i],p,r);
    for i in range(int(N*gamma),N):
        Y[i]= Belief(B[i],q,s);
    
    Y.sort();
    Threshold = Y[int((1-alpha)*N)];
    
    for i in range(int(N*gamma)):
        BeliefVector[i]= Belief(B[i],p,r);
    for i in range(int(N*gamma),N):
        BeliefVector[i]= Belief(B[i],q,s);
    
    #Get the Action vector.    
    h=0;
    for i in range(N):
        if BeliefVector[i]>= Threshold:
            Action[i]=1;
            h=h+1;
        if h >= alpha*N:
            break;
    
    #The update process.        

    for i in range(int(N*gamma)):
        
            if Action[i] == 0 and B[i][1]>0:
                B[i][1]= B[i][1]+1;
                if C[i]==0:
                    if Seed[i]<r:
                        C[i]=1;
                    else:
                        C[i]=0;
                else:
                    if Seed[i]<p:
                        C[i]=1;
                    else:
                        C[i]=0;
                
                
                
               
             
            elif Action[i] == 0 and B[i][1]==0:
                if C[i]==0:
                    if Seed[i]<r:
                        C[i]=1;
                    else:
                        C[i]=0;
                else: 
                    if Seed[i]<p:
                        C[i]=1;
                    else:
                        C[i]=0;
            
            else:
                if C[i]==1:
                    B[i]= [1,1];
                    Reward = Reward + 1;
                    C[i]= int((Seed[i]<p)==True);
                    
                else:
                    B[i]= [0,1];
                    C[i]= int((Seed[i]<r)==True);
        
    for i in range(int(N*gamma),N):
        
            if Action[i] == 0 and B[i][1]>0:
                B[i][1]= B[i][1]+1;
                if C[i]==0:
                    if Seed[i]<s:
                        C[i]=1;
                    else:
                        C[i]=0;
                else:
                    if Seed[i]<q:
                        C[i]=1;
                    else:
                        C[i]=0;
                
                
                
               
             
            elif Action[i] == 0 and B[i][1]==0:
                if C[i]==0:
                    if Seed[i]<s:
                        C[i]=1;
                    else:
                        C[i]=0;
                else: 
                    if Seed[i]<q:
                        C[i]=1;
                    else:
                        C[i]=0;
            
            else:
                if C[i]==1:
                    B[i]= [1,1];
                    Reward = Reward + 1;
                    C[i]= int((Seed[i]<q)==True);
                    
                else:
                    B[i]= [0,1];
                    C[i]= int((Seed[i]<s)==True);
        
        
                    
    return Reward;
    
    
    
    
    

In [2]:
print(update([[1,2],[0,1],[0,3],[0,2],[0,0],[1,4],[1,3],[1,7],[0,4],[0,6]],[0,0,1,0,0,1,1,0,0,0],0,0.5,0.4,0.3,0.6,0.2,0.6,10));

2


In [3]:
def onetrial(B,C,Reward,timesteps,alpha,p,r,q,s,gamma,N):

    for i in range(timesteps):
        Reward = update(B,C,Reward,alpha,p,r,q,s,gamma,N);
    return Reward;

In [4]:
print(onetrial([[1,2],[0,1],[0,3],[0,2],[0,0],[1,4],[1,3],[1,7],[0,4],[0,6]],[0,0,1,0,0,1,1,0,0,0],0,100,0.5,0.4,0.3,0.6,0.2,0.6,10));

179


In [5]:
def mc(B,Reward,ntrials,timesteps,alpha,p,r,q,s,gamma,N):
    e_1 = r/(1-p+r);
    e_2 = s/(1-q+s);
    
    bigB = [B]*(ntrials);
    for i in range(ntrials):
        C = [0]*N;
        Seed = np.random.rand(N);
        for j in range(int(N*gamma)):
            C[j]= int((Seed[j]< e_1) == True);
        for j in range(int(N*gamma),N):
            C[j]= int((Seed[j]< e_2) == True);
        Reward = onetrial(bigB[i],C,Reward,timesteps,alpha,p,r,q,s,gamma,N);
        
    lowerbound = 100*Reward/(timesteps*ntrials*N);
    
    print(lowerbound);


In [12]:
def bg(N):
    bg= mc([[0,0]]*N,0,500,1000,0.5,0.4,0.3,0.6,0.2,0.6,N);
    return bg;

y= [bg(10),bg(20),bg(30),bg(40),bg(50),bg(100),bg(200)];
    

18.26048
18.3741
18.464
18.56067
18.623748
18.785422
18.829368


In [14]:
def bg(N):
    bg= mc([[0,0]]*N,0,500,1000,0.7,0.9,1/15,0.5,1/3,0.5,N);
    return bg;

y= [bg(10),bg(20),bg(30),bg(40),bg(50),bg(100),bg(200)];
    

34.1174
34.21696
34.297673333333336
34.410165
34.509508
34.68941
34.830054


In [15]:
def bg(N):
    bg= mc([[0,0]]*N,0,500,1000,0.2,0.65,0.15,0.44,0.24,0.5,N);
    return bg;

y= [bg(10),bg(20),bg(30),bg(40),bg(50),bg(100),bg(200)];

7.77304
7.7425
7.7432533333333335
7.73471
7.740276
7.744882
7.746615


In [6]:
def bg(N):
    bg= mc([[0,0]]*N,0,500,1000,0.2,0.9,1/15,0.5,1/3,0.5,N);
    return bg;

y= [bg(10),bg(20),bg(30),bg(40),bg(50),bg(100),bg(200)];
    

10.93066
10.76353
10.72564
10.70337
10.682688
10.651726
10.642225
